### Import libraries

In [2]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
import numpy as np
import pandas as pd

### Load data into dataframe

In [3]:
df= pd.read_csv("data.csv")

df.head()

,Ca,GPP,Ndep,Pr,SR,Ta
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN


### Drop all null values

In [4]:
df=df.dropna()
df.head()

,Ca,GPP,Ndep,Pr,SR,Ta
342,368.819726,974.160673,185.570690,507.712412,142.448980,267.149766
343,368.819726,962.588002,186.690840,489.440898,141.267288,267.434053
344,368.819726,957.275438,187.812488,499.614568,139.873844,267.309729
345,368.819726,981.238993,188.936788,508.315295,138.695632,267.830984
346,368.819726,1013.469566,191.204997,503.878858,137.707015,268.769265


### Get an overview of the data

In [26]:
df.describe()

,Ca,GPP,Ndep,Pr,SR,Ta
count,2229.000000,2229.000000,2229.000000,2229.000000,2229.000000,2229.000000
mean,368.674281,688.769640,932.006627,576.800122,175.638873,280.417778
std,0.047611,620.929154,848.165248,478.261483,27.044387,6.967650
min,368.618885,4.081976,79.870266,29.509232,116.788359,262.675793
25%,368.630420,99.743677,288.035417,175.539849,152.008991,275.239474
50%,368.660854,538.249208,587.708287,490.134209,176.300341,280.684978
75%,368.720942,1102.688495,1357.226063,772.589152,197.201660,285.810076
max,368.819726,2593.126339,4550.184428,2054.276437,262.276507,295.062065


### Set the target and split dataset into train, validation, and test datasets

In [25]:
target= "GPP"
X= df.drop(columns=[target])
y= df[target]

# No need to stratify since our data is balanced
# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Split the training set into training and validation sets for hyperparameter tuning
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)

print(f"train amount: {len(X_train)}")
print(f"validation amount: {len(X_val)}")
print(f"test amount: {len(X_test)}")


train amount: 1253
validation amount: 418
test amount: 558


### Try a simple method of training the model

In [61]:
simple_model = XGBRegressor(
    n_estimators=10000, early_stopping_rounds=50, random_state=42, n_jobs=-1
)
simple_model.fit(X_train, y_train, verbose=False, eval_set=[(X_val, y_val)])

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,50
,enable_categorical,False
,eval_metric,None


### Test the R² score

In [63]:
from sklearn.metrics import mean_squared_error

def test_model_performance(model, X_test, y_test):
    r2 = model.score(X_test, y_test)
    mse = mean_squared_error(y_test, model.predict(X_test))
    print("R² on test set: ", r2)
    print("MSE: ", mse)

In [62]:
test_model_performance(simple_model, X_test, y_test)

R² on test set:  0.9710001880770307
MSE:  10357.558238821395


### Define the model (XGBRegressor for continuous values)

In [78]:
gbtreeModel = XGBRegressor(
    n_estimators=10000,
    early_stopping_rounds=50,
    n_jobs=-1,
    random_state=46,
    tree_method="hist",
)

dartModel = XGBRegressor(
    n_estimators=1000,
    early_stopping_rounds=50,
    n_jobs=-1,
    random_state=46,
    tree_method="hist",
)

### XGBRegressor boosters
`gbtree`, `dart`, or `gblinear`?

#### gbtree — Gradient-boosted decision trees
- Best for: Most structured/tabular problems
- Strengths: Captures nonlinearities and interactions; robust; handles missing values
- Key params: max_depth, min_child_weight, subsample, colsample_bytree, gamma, reg_alpha, reg_lambda, n_estimators, learning_rate
- Notes: Use tree_method=hist/gpu_hist

#### dart — Trees with dropout (regularized gbtree)
- Best for: When gbtree overfits or validation metric is unstable
- Strengths: Often improves generalization vs gbtree
- Trade-offs: Slower, less deterministic; more hyperparameters
- Extra params: rate_drop, skip_drop, sample_type, normalize_type, one_drop
- Notes: Same tree params as gbtree; tree_method applies

#### gblinear — Linear booster (GLM with L1/L2)
- Best for: High-dimensional sparse data or problems known to be linear
- Strengths: Very fast; simple and interpretable
- Limitations: Only linear relationships unless you engineer features
- Key params: reg_alpha (L1), reg_lambda (L2), updater, feature_selector; learning_rate/n_estimators still matter
- Notes: tree_method is ignored

### Define the hyperparameters for model tuning
Check the notes in README.md for more information on the parameters

[Official param docs](https://xgboost.readthedocs.io/en/stable/parameter.html)

### DART (Dropouts meet Multiple Additive Regression Trees)

In [79]:
param_distributions = {
    # Fixed core params based on your GBTree results
    "booster": ["dart"],
    "learning_rate": [0.08, 0.1, 0.12],          # Around your best (0.102)
    "max_depth": [8],                            # Fix at your best value
    "min_child_weight": [6, 7, 8],               # Around your best (7)
    "subsample": [0.6],                          # Fix near your best (0.598)
    "colsample_bytree": [0.85, 0.9],             # Around your best (0.878)
    "alpha": [7],                                # Fix at your best value
    "lambda": [40, 50, 60],                      # Around your best (50)
    
    # DART-specific params to explore (these matter most for DART)
    "rate_drop": [0.04, 0.05, 0.06],
}
# Configure the randomized search
random_search_1 = RandomizedSearchCV(
    estimator=dartModel,
    param_distributions=param_distributions,
    n_iter=10,                                 # Reduced iterations
    cv=3,                                      # Keep CV=3
    scoring="r2",
    n_jobs=-1,
    random_state=46,
)

# Search for the best hyperparameters
random_search_1.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

,estimator,"XGBRegressor(...ree=None, ...)"
,param_distributions,"{'alpha': [7], 'booster': ['dart'], 'colsample_bytree': [0.85, 0.9], 'lambda': [40, 50, ...], ...}"
,n_iter,10
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,3
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,46
,error_score,nan


### (Dart) Print the best set of hyperparameters and the corresponding score

In [80]:
print(f"Best set of hyperparameters: ", random_search_1.best_params_)
test_model_performance(random_search_1, X_test, y_test)

Best set of hyperparameters:  {'subsample': 0.6, 'rate_drop': 0.04, 'min_child_weight': 7, 'max_depth': 8, 'learning_rate': 0.08, 'lambda': 40, 'colsample_bytree': 0.9, 'booster': 'dart', 'alpha': 7}
R² on test set:  0.950054548943916
MSE:  17838.49217545635


### (Dart) Train the model based on the optimal hyperparameters

In [ ]:
dart_estimator_model=random_search_1.best_estimator_
dart_optimal_params = dart_estimator_model.get_params()
dart_optimal_params["n_estimators"] = 10000
dart_optimal_params["early_stopping_rounds"] = 50
dart_tuned_model = XGBRegressor(**dart_optimal_params)
dart_tuned_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
test_model_performance(dart_tuned_model,  X_test, y_test)

R² on test set:  0.9554620047682009
MSE:  15907.167973332034


### GBTree (Gradient Boosted Trees)

In [99]:
import scipy

param_distributions = {
    "learning_rate": scipy.stats.uniform(loc=0.003, scale=0.19),  # Default is 0.3. Ranges from loc to loc+scale.
    "subsample": scipy.stats.uniform(loc=0.5, scale=0.5),  # Default is 1
    "colsample_bytree": scipy.stats.uniform(loc=0.5, scale=0.5),  # Default is 1
    "min_child_weight": [1, 3, 5, 7],  # Default is 1
    "max_depth": np.append(0, np.arange(3, 16)),  # Default is 6
    "alpha": [0, 0.01, 1, 2, 5, 7, 10, 50, 100],  # Default is 0. AKA reg_alpha.
    "lambda": [0, 0.01, 1, 5, 10, 20, 50, 100]  # Default is 0. AKA reg_lambda.
}
# Configure the randomized search
random_search_2 = RandomizedSearchCV(
    estimator=gbtreeModel,
    param_distributions=param_distributions,
    n_iter=40,
    cv=3,
    scoring="r2",
    n_jobs=-1,
    random_state=46,
)

# Search for the best hyperparameters
random_search_2.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

print(f"Best set of hyperparameters: ", random_search_2.best_params_)
print(f"Best score: {random_search_2.best_score_}")

Best set of hyperparameters:  {'alpha': 7, 'colsample_bytree': np.float64(0.8778327171409642), 'lambda': 50, 'learning_rate': np.float64(0.10170070441506318), 'max_depth': np.int64(8), 'min_child_weight': 7, 'subsample': np.float64(0.5975636483724356)}
Best score: 0.9691891108761469


### (GBTree) Print the best set of hyperparameters and the corresponding score

In [85]:
test_model_performance(random_search_2, X_test, y_test)

R² on test set:  0.9764923769849196
MSE:  8395.970811179779


### (GBTree) Train the model based on the optimal hyperparameters

In [106]:
gbtree_estimator_model=random_search_2.best_estimator_
gbtree_optimal_params = gbtree_estimator_model.get_params()
gbtree_optimal_params["n_estimators"] = gbtree_estimator_model.best_iteration
gbtree_optimal_params["early_stopping_rounds"] = 50
gbTree_tuned_model = XGBRegressor(**gbtree_optimal_params)
gbTree_tuned_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
test_model_performance(gbTree_tuned_model, X_test, y_test)

R² on test set:  0.9764872954780774
MSE:  8397.78571960741


### Save the models (gbTree performed better)

In [110]:
gbTree_tuned_model.save_model("gbtree_model.bin")
dart_tuned_model.save_model("dart_model.bin")

# Save the best model
gbTree_tuned_model.save_model("best_model.bin")

/opt/homebrew/Caskroom/miniconda/base/envs/xgboost-gpp/lib/python3.13/site-packages/xgboost/sklearn.py:1028: UserWarning: [22:37:47] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1755048541311/work/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
